# 1. Parameters

In [1]:
# Defaults
cases_dir = 'cases/unset'
reference_file = 'references/NC_045512.gbk.gz'
input_files_all = 'input/input-files.tsv'
iterations = 3
mincov = 10
ncores = 32
number_samples = 10
build_tree = False
sample_batch_size=2000

In [2]:
# Parameters
cases_dir = "cases/case-20000"
iterations = 3
number_samples = 20000
build_tree = False


In [3]:
from pathlib import Path
from shutil import rmtree
from os import makedirs
import imp
fp, pathname, description = imp.find_module('gdi_benchmark', ['../../lib'])
gdi_benchmark = imp.load_module('gdi_benchmark', fp, pathname, description)

cases_dir_path = Path(cases_dir)

if cases_dir_path.exists():
    rmtree(cases_dir_path)
    
if not cases_dir_path.exists():
    makedirs(cases_dir_path)

input_files_all = Path(input_files_all)
reference_file = Path(reference_file)

case_name = str(cases_dir_path.name)
reference_name = reference_file.name.split('.')[0]

cases_input = cases_dir_path / 'input-files-case.tsv'
index_path = cases_dir_path / 'index'
benchmark_path = cases_dir_path / 'index-info.tsv'
output_tree = cases_dir_path / 'tree.tre'

# 2. Create subset input

In [4]:
import pandas as pd

all_input_df = pd.read_csv(input_files_all, sep='\t')
all_input_total = len(all_input_df)

subset_input_df = all_input_df.head(number_samples)
subset_input_total = len(subset_input_df)

subset_input_df.to_csv(cases_input, sep='\t', index=False)

print(f'Wrote {subset_input_total}/{all_input_total} samples to {cases_input}')

Wrote 20000/100000 samples to cases/case-20000/input-files-case.tsv


# 2. Index genomes

In [5]:
!gdi --version

gdi, version 0.6.0.dev2


## 2.1. Index reads

In [6]:
results_handler = gdi_benchmark.BenchmarkResultsHandler(name=case_name)
benchmarker = gdi_benchmark.IndexBenchmarker(benchmark_results_handler=results_handler,
                                             index_path=index_path, input_files_file=cases_input,
                                             reference_file=reference_file, mincov=mincov,
                                             build_tree=build_tree,
                                             ncores=ncores,
                                             sample_batch_size=sample_batch_size)

benchmark_df = benchmarker.benchmark(iterations=iterations)


Iteration 1 of index/analysis of 20000 samples with 32 cores
Removing any extra snakemake directories: ['snakemake-assemblies.1646960020.5148413']


Creating new index: [gdi init cases/case-20000/index]


Creating a new index took 2.84 seconds
Analysis running: [gdi --project-dir cases/case-20000/index --ncores 32 analysis --use-conda --no-load-data --reference-file references/NC_045512.gbk.gz --kmer-size 31 --kmer-size 51 --kmer-size 71 --include-kmer --reads-mincov 10 --input-structured-genomes-file cases/case-20000/input-files-case.tsv]


Analysis took 130.95 minutes
Index running: [gdi --project-dir cases/case-20000/index --ncores 32 load vcf-kmer --sample-batch-size 2000 --reference-file references/NC_045512.gbk.gz /home/CSCScience.ca/apetkau/workspace/genomics-data-index-evaluation/evaluations/sars-cov-2/snakemake-assemblies.1646964575.6465294/gdi-input.fofn]


Indexing took 21.81 minutes


...

Creating a new index took 3.24 seconds
Analysis running: [gdi --project-dir cases/case-20000/index --ncores 32 analysis --use-conda --no-load-data --reference-file references/NC_045512.gbk.gz --kmer-size 31 --kmer-size 51 --kmer-size 71 --include-kmer --reads-mincov 10 --input-structured-genomes-file cases/case-20000/input-files-case.tsv]


Analysis took 130.01 minutes
Index running: [gdi --project-dir cases/case-20000/index --ncores 32 load vcf-kmer --sample-batch-size 2000 --reference-file references/NC_045512.gbk.gz /home/CSCScience.ca/apetkau/workspace/genomics-data-index-evaluation/evaluations/sars-cov-2/snakemake-assemblies.1646982984.1068096/gdi-input.fofn]


Indexing took 23.26 minutes


Attempting to set global database_path_translator=<genomics_data_index.storage.model.db.DatabasePathTranslator.DatabasePathTranslator object at 0x7fc120ddaee0> but it is already set


A reminder to myself to look for a Python solution for directory sizes (instead of running `du`)


In [7]:
benchmark_df

,Name,Reference name,Iteration,Number samples,Number features (all),Number features (no unknown),Number cores,Reference length,Analysis runtime,Analysis memory (max),...,Analysis disk uage,Index runtime,Index memory (max),Index memory (max/process),Index size,Tree runtime,Tree memory (max),Tree memory (max/process),Total runtime,Max memory
0,case-20000,NC_045512,1,20000,45709,15806,32,29903,7805.0,2.195067e+09,...,7.628685e+09,1308.62,3.585326e+10,2.400236e+09,365961216.0,<NA>,<NA>,<NA>,9113.62,3.585326e+10
0,case-20000,NC_045512,2,20000,45709,15806,32,29903,7801.0,2.192609e+09,...,7.628861e+09,1393.05,3.498883e+10,2.360087e+09,365944832.0,<NA>,<NA>,<NA>,9194.05,3.498883e+10
0,case-20000,NC_045512,3,20000,45709,15806,32,29903,7800.0,2.199642e+09,...,7.628517e+09,1395.52,3.566413e+10,2.389336e+09,366084096.0,<NA>,<NA>,<NA>,9195.52,3.566413e+10


In [8]:
benchmark_df.to_csv(benchmark_path, sep='\t', index=False)

# 3. Export trees

In [9]:
if build_tree:
    !gdi --project-dir {index_path} export tree {reference_name} > {output_tree}
    print(f'Wrote tree to {output_tree}')
else:
    print(f'build_tree={build_tree} so no tree to export')

build_tree=False so no tree to export
